In [ ]:
import yfinance as yf
import pandas as pd

def fetch_and_transform_data(ticker_symbol, statement_type):
    try:
        # Fetch financial data based on the specified statement type
        if statement_type == 'CF':
            financial_data = yf.Ticker(ticker_symbol).cashflow
        elif statement_type == 'IS':
            financial_data = yf.Ticker(ticker_symbol).financials
        elif statement_type == 'BS':
            financial_data = yf.Ticker(ticker_symbol).balancesheet

        # Prepare data for transformation
        dfs = []
        available_dates = financial_data.columns
        
        # Iterate over each available date
        for date in available_dates:
            # Extract financial data for the current date
            year = pd.to_datetime(date).year

            # Filter data to only include years from 2015 onwards
            if year == 2015:
                data_date = financial_data[date]
                
                # Append each account as a row in the DataFrame
                for account, value in data_date.items():
                    dfs.append({
                        'symbol': ticker_symbol,
                        'account': account,
                        'type': statement_type,  
                        'year': year,
                        'value': value
                    })
        
        # Create DataFrame from the collected data
        return pd.DataFrame(dfs)
    
    except Exception as e:
        print(f"No data available for {ticker_symbol} ({statement_type}): {e}")
        return None

# List of ticker symbols to process
ticker_symbols = ['AALI.JK', 'BBCA.JK', 'TLKM.JK']  # Replace with your actual ticker symbols

# Fetch and combine data for all ticker symbols
combined_dataframes = []

for ticker_symbol in ticker_symbols:
    # Fetch and transform cash flow data
    cash_flow_data = fetch_and_transform_data(ticker_symbol, 'CF')
    
    # Fetch and transform income statement data
    income_statement_data = fetch_and_transform_data(ticker_symbol, 'IS')

    # Fetch and transform balance sheet data
    balance_sheet_data = fetch_and_transform_data(ticker_symbol, 'BS')

    # Concatenate dataframes (cash flow, income statement, and balance sheet) for the current ticker symbol
    combined_data = pd.concat([cash_flow_data, income_statement_data, balance_sheet_data], ignore_index=True)
    combined_dataframes.append(combined_data)

# Combine all dataframes into a single DataFrame
if combined_dataframes:
    all_data = pd.concat(combined_dataframes, ignore_index=True)

    # Pivot the DataFrame to convert dates into columns
    pivot_data = all_data.pivot_table(index=['symbol', 'account', 'type'], columns='year', values='value').reset_index()

    # Reorder columns to match the desired format
    desired_columns_order = ['symbol', 'account', 'type'] + list(pivot_data.columns[3:])  # Include all columns after 'type'
    
    final_data = pivot_data[desired_columns_order]
    final_data['symbol'] = final_data['symbol'].apply(lambda x: x.split('.')[0])

    # Convert NaN values to empty strings if needed
    final_data = final_data.fillna('')

    # Save DataFrame to CSV
    filename = 'combined_financial_data_idx.csv'
    final_data.to_csv(filename, index=False)

    # Display the final transformed data
    print("Combined financial data:")
    print(final_data)
else:
    print("Unable to retrieve and combine financial data for all ticker symbols.")
